In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.models.video import r3d_18
from torchvision import transforms
import cv2
from PIL import Image
import os


In [2]:
class PreloadedDataset(Dataset):
    def __init__(self, root_dir, categories, sequence_length=16, transform=None):
        """
        Args:
            root_dir (str): Root directory containing category subfolders.
            categories (list): List of category folder names.
            sequence_length (int): Number of frames per sequence.
            transform (callable): Transformations to apply to each frame.
        """
        self.data = []  # Preloaded frames
        self.labels = []  # Corresponding labels
        self.sequence_length = sequence_length
        self.transform = transform

        # Preload all frames into memory
        for label, category in enumerate(categories):
            category_path = os.path.join(root_dir, category)
            frame_files = sorted([f for f in os.listdir(category_path) if f.endswith(".png")])
            print(f"Loading category: {category} ({len(frame_files)} frames)")

            for frame in frame_files:
                img = Image.open(os.path.join(category_path, frame)).convert("RGB")
                if self.transform:
                    img = self.transform(img)
                self.data.append(img)
                self.labels.append(label)

    def __len__(self):
        # Divide total frames by sequence length
        return len(self.data) // self.sequence_length

    def __getitem__(self, idx):
        start_idx = idx * self.sequence_length
        end_idx = start_idx + self.sequence_length

        # Get the sequence of frames
        sequence = self.data[start_idx:end_idx]
        label = self.labels[start_idx]  # Use the label of the first frame in the sequence

        # Stack frames into tensor: (C, T, H, W)
        video_tensor = torch.stack(sequence, dim=1)
        return video_tensor, label

In [3]:
# Define categories and paths
categories = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", 
              "Fighting", "RoadAccidents", "Robbery", "Shooting", 
              "Shoplifting", "Stealing", "Vandalism"]
train_root = "Train"
test_root = "Test"

# Define transforms
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize to match model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize
])


In [4]:
# Preload train dataset
sequence_length = 16
train_dataset = PreloadedDataset(train_root, categories, sequence_length, transform)

# Preload test dataset
test_dataset = PreloadedDataset(test_root, categories, sequence_length, transform)

# Split train dataset into train and validation sets
from torch.utils.data import random_split, DataLoader
val_split = 0.2
val_size = int(len(train_dataset) * val_split)
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Loading category: Abuse (19076 frames)
Loading category: Arson (24421 frames)
Loading category: Assault (10360 frames)
Loading category: Burglary (39504 frames)
Loading category: Explosion (18753 frames)
Loading category: Fighting (24684 frames)
Loading category: RoadAccidents (23486 frames)
Loading category: Robbery (41493 frames)
Loading category: Shooting (7140 frames)
Loading category: Shoplifting (24835 frames)
Loading category: Stealing (44802 frames)
Loading category: Vandalism (13626 frames)
Loading category: Abuse (297 frames)
Loading category: Arson (2793 frames)
Loading category: Assault (2657 frames)
Loading category: Burglary (7657 frames)
Loading category: Explosion (6510 frames)
Loading category: Fighting (1231 frames)
Loading category: RoadAccidents (2663 frames)
Loading category: Robbery (835 frames)
Loading category: Shooting (7630 frames)
Loading category: Shoplifting (7623 frames)
Loading category: Stealing (1984 frames)
Loading category: Vandalism (1111 frames)


In [5]:
# Load pretrained ResNet3D model
num_classes = len(categories)
model = r3d_18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace the final layer
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


C:\Users\adars\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\adars\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Print all layers with their names
for name, param in model.named_parameters():
    print(name)


stem.0.weight
stem.1.weight
stem.1.bias
layer1.0.conv1.0.weight
layer1.0.conv1.1.weight
layer1.0.conv1.1.bias
layer1.0.conv2.0.weight
layer1.0.conv2.1.weight
layer1.0.conv2.1.bias
layer1.1.conv1.0.weight
layer1.1.conv1.1.weight
layer1.1.conv1.1.bias
layer1.1.conv2.0.weight
layer1.1.conv2.1.weight
layer1.1.conv2.1.bias
layer2.0.conv1.0.weight
layer2.0.conv1.1.weight
layer2.0.conv1.1.bias
layer2.0.conv2.0.weight
layer2.0.conv2.1.weight
layer2.0.conv2.1.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.0.weight
layer2.1.conv1.1.weight
layer2.1.conv1.1.bias
layer2.1.conv2.0.weight
layer2.1.conv2.1.weight
layer2.1.conv2.1.bias
layer3.0.conv1.0.weight
layer3.0.conv1.1.weight
layer3.0.conv1.1.bias
layer3.0.conv2.0.weight
layer3.0.conv2.1.weight
layer3.0.conv2.1.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.0.weight
layer3.1.conv1.1.weight
layer3.1.conv1.1.bias
layer3.1.conv2.0.wei

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [7]:
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import os

# Initialize GradScaler
scaler = GradScaler()

# Ensure save directory exists
save_dir = "checkpoints"
os.makedirs(save_dir, exist_ok=True)

num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(1, num_epochs + 1):
    print(f"Starting Epoch {epoch}/{num_epochs}")
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop with tqdm
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Mixed precision forward pass
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Scaled backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())

    train_accuracy = train_correct / train_total
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

    # Validation loop with tqdm
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    progress_bar = tqdm(val_loader, desc=f"Validating Epoch {epoch}")
    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            # Mixed precision forward pass
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

            # Update progress bar
            progress_bar.set_postfix(loss=loss.item())

    val_accuracy = val_correct / val_total
    val_loss /= len(val_loader)
    print(f"Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

    # Save the model checkpoint
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch,
    }
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved at {save_path}")

    # Step scheduler
    scheduler.step()

print("Training complete.")


C:\Users\adars\AppData\Local\Temp\ipykernel_25488\226602884.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Starting Epoch 1/20


Training Epoch 1:   0%|          | 0/457 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_25488\226602884.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 1: 100%|██████████| 457/457 [05:17<00:00,  1.44it/s, loss=0.352] 


Epoch 1: Train Loss = 0.4603, Train Accuracy = 0.8651


Validating Epoch 1:   0%|          | 0/115 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_25488\226602884.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validating Epoch 1: 100%|██████████| 115/115 [00:48<00:00,  2.35it/s, loss=0.0753]


...

Epoch 7: Train Loss = 0.0020, Train Accuracy = 0.9997


Validating Epoch 7: 100%|██████████| 115/115 [00:47<00:00,  2.40it/s, loss=6.6e-5]  


Validation Loss = 0.0201, Validation Accuracy = 0.9948
Checkpoint saved at checkpoints\checkpoint_epoch_7.pth
Starting Epoch 8/20


Training Epoch 8:   3%|▎         | 15/457 [00:10<05:02,  1.46it/s, loss=0.000379]


KeyboardInterrupt: 

FINETUNING FC LAYER

In [20]:
# Load pretrained ResNet3D model
num_classes = len(categories)  # Replace with your number of classes
model = r3d_18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace final layer

# Optional: Freeze all layers except `layer4` and `fc` (fine-tuning example)
for param in model.parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

# Ensure save directory exists
save_dir = "checkpoints2"
os.makedirs(save_dir, exist_ok=True)

# Training configuration
num_epochs = 5

print(f"Using device: {device}")

Using device: cuda


C:\Users\adars\AppData\Local\Temp\ipykernel_25488\544432743.py:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [21]:
for epoch in range(1, num_epochs + 1):
    print(f"Starting Epoch {epoch}/{num_epochs}")
    
    # Training loop
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Mixed precision forward pass
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Scaled backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

        # Update progress bar
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    train_accuracy = train_correct / train_total
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    progress_bar = tqdm(val_loader, desc=f"Validating Epoch {epoch}")

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            # Mixed precision forward pass
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            # Update progress bar
            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    val_accuracy = val_correct / val_total
    val_loss /= len(val_loader)
    print(f"Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

    # Save the model checkpoint
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch,
    }
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved at {save_path}")

    # Step scheduler
    scheduler.step()

print("Training complete.")

Starting Epoch 1/5


Training Epoch 1:   0%|          | 0/457 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_25488\3227304310.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 1: 100%|██████████| 457/457 [03:12<00:00,  2.38it/s, loss=0.3508]


Epoch 1: Train Loss = 0.3283, Train Accuracy = 0.9047


Validating Epoch 1:   0%|          | 0/115 [00:00<?, ?it/s]C:\Users\adars\AppData\Local\Temp\ipykernel_25488\3227304310.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validating Epoch 1: 100%|██████████| 115/115 [00:48<00:00,  2.38it/s, loss=0.0072]


Validation Loss = 0.0650, Validation Accuracy = 0.9838
Checkpoint saved at checkpoints2\checkpoint_epoch_1.pth
Starting Epoch 2/5


...

Validation Loss = 0.0561, Validation Accuracy = 0.9830
Checkpoint saved at checkpoints2\checkpoint_epoch_4.pth
Starting Epoch 5/5


Training Epoch 5: 100%|██████████| 457/457 [03:10<00:00,  2.39it/s, loss=0.0314]


Epoch 5: Train Loss = 0.0325, Train Accuracy = 0.9905


Validating Epoch 5: 100%|██████████| 115/115 [00:54<00:00,  2.12it/s, loss=0.0000]


Validation Loss = 0.0422, Validation Accuracy = 0.9885
Checkpoint saved at checkpoints2\checkpoint_epoch_5.pth
Training complete.


3RD LAYER TRAINING

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models.video import r3d_18
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

# Load pretrained ResNet3D model
num_classes = len(categories)  # Replace with your number of classes
model = r3d_18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze `layer3`, `layer4`, and `fc`
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True

# Verify trainable layers
print("Trainable layers:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define criterion, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW([
    {'params': model.fc.parameters(), 'lr': 0.001},          # Higher LR for final layer
    {'params': model.layer4.parameters(), 'lr': 0.0001},    # Lower LR for layer4
    {'params': model.layer3.parameters(), 'lr': 0.00005},   # Even lower LR for layer3
])

scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Training configuration
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    print(f"Starting Epoch {epoch}/{num_epochs}")
    
    # Training loop
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

        # Update progress bar
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    train_loss = running_loss / len(train_loader)
    train_accuracy = train_correct / train_total
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    progress_bar = tqdm(val_loader, desc=f"Validating Epoch {epoch}")

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            # Update progress bar
            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    val_loss /= len(val_loader)
    val_accuracy = val_correct / val_total
    print(f"Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

    # Save checkpoint
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
    }
    torch.save(checkpoint, f"checkpoint_epoch_{epoch}.pth")

    # Step scheduler
    scheduler.step()

print("Training complete.")


Trainable layers:
layer3.0.conv1.0.weight
layer3.0.conv1.1.weight
layer3.0.conv1.1.bias
layer3.0.conv2.0.weight
layer3.0.conv2.1.weight
layer3.0.conv2.1.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.0.weight
layer3.1.conv1.1.weight
layer3.1.conv1.1.bias
layer3.1.conv2.0.weight
layer3.1.conv2.1.weight
layer3.1.conv2.1.bias
layer4.0.conv1.0.weight
layer4.0.conv1.1.weight
layer4.0.conv1.1.bias
layer4.0.conv2.0.weight
layer4.0.conv2.1.weight
layer4.0.conv2.1.bias
layer4.0.downsample.0.weight
layer4.0.downsample.1.weight
layer4.0.downsample.1.bias
layer4.1.conv1.0.weight
layer4.1.conv1.1.weight
layer4.1.conv1.1.bias
layer4.1.conv2.0.weight
layer4.1.conv2.1.weight
layer4.1.conv2.1.bias
fc.weight
fc.bias
Starting Epoch 1/20


Training Epoch 1: 100%|██████████| 457/457 [05:17<00:00,  1.44it/s, loss=0.0285]


Epoch 1: Train Loss = 0.3575, Train Accuracy = 0.8988


Validating Epoch 1: 100%|██████████| 115/115 [01:09<00:00,  1.65it/s, loss=0.0011]


Validation Loss = 0.0487, Validation Accuracy = 0.9882
Starting Epoch 2/20


...

Epoch 10: Train Loss = 0.0002, Train Accuracy = 1.0000


Validating Epoch 10: 100%|██████████| 115/115 [01:02<00:00,  1.85it/s, loss=0.0000]


Validation Loss = 0.0291, Validation Accuracy = 0.9929
Starting Epoch 11/20


Training Epoch 11:  52%|█████▏    | 237/457 [02:21<02:11,  1.67it/s, loss=0.0002]


KeyboardInterrupt: 

TO CONTINUE TRAINING

In [ ]:
# Path to the saved checkpoint
checkpoint_path = "checkpoints\checkpoint_epoch_7.pth"  # Replace with your saved checkpoint file

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Reinitialize the model
model = r3d_18(pretrained=True)  # Ensure to use the same model architecture
model.fc = nn.Linear(model.fc.in_features, 12)  # Replace with your number of classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load the model weights
model.load_state_dict(checkpoint['model_state_dict'])

# Reinitialize the optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

# Start from the saved epoch
start_epoch = checkpoint['epoch'] + 1
print(f"Resuming training from epoch {start_epoch}")


In [ ]:
num_epochs = 20  # Total number of epochs to train (including previously completed epochs)
save_dir = "models"

for epoch in range(start_epoch, num_epochs + 1):
    print(f"Starting Epoch {epoch}/{num_epochs}")
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    val_loss /= len(val_loader)
    print(f"Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.4f}")

    # Save the checkpoint after each epoch
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch
    }, f"checkpoint_epoch_{epoch}.pth")

    # Step the scheduler
    scheduler.step()

print("Training complete.")


MODEL EVALUATION

In [26]:
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = test_correct / test_total
test_loss /= len(test_loader)
print(f"Test Loss = {test_loss:.4f}, Test Accuracy = {test_accuracy:.4f}")


Test Loss = 4.5756, Test Accuracy = 0.1772


VIDEO INFERENCE CATEGORY


In [25]:
import torch
from torchvision.models.video import r3d_18
import torch.nn as nn

# Load the saved checkpoint
checkpoint_path = "checkpoint_epoch_10.pth"  # Replace with the path to your checkpoint
checkpoint = torch.load(checkpoint_path)

# Reinitialize the model with the same architecture used during training
num_classes = 12  # Replace with the number of classes in your dataset
model = r3d_18(pretrained=False)  # Ensure pretrained=False as you're loading trained weights
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace final layer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load the trained weights into the model
model.load_state_dict(checkpoint['model_state_dict'])

# Set the model to evaluation mode
model.eval()
print("Model loaded and ready for inference.")


C:\Users\adars\AppData\Local\Temp\ipykernel_25488\1342778880.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Model loaded and ready for inference.


In [13]:
import cv2
from torchvision import transforms
from PIL import Image

def preprocess_video(video_path, sequence_length=16):
    """
    Preprocess a video file into a tensor suitable for the ResNet3D model.

    Args:
        video_path (str): Path to the video file.
        sequence_length (int): Number of frames to extract.

    Returns:
        torch.Tensor: Preprocessed video tensor of shape (1, C, T, H, W).
    """
    transform = transforms.Compose([
        transforms.Resize((112, 112)),  # ResNet3D expects frames of size 112x112
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])
    
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while cap.isOpened() and frame_count < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frame = Image.fromarray(frame)  # Convert to PIL image
        frame = transform(frame)  # Apply transformations
        frames.append(frame)
        frame_count += 1

    cap.release()

    # Pad with black frames if the video has fewer frames than sequence_length
    while len(frames) < sequence_length:
        frames.append(torch.zeros_like(frames[0]))

    # Stack frames into a single tensor: (Batch_Size=1, Channels, Frames, Height, Width)
    video_tensor = torch.stack(frames, dim=1).unsqueeze(0)
    return video_tensor


In [14]:
def predict_video(model, video_tensor, device):
    """
    Predict the class of a video using the trained model.

    Args:
        model (nn.Module): The trained model.
        video_tensor (torch.Tensor): Preprocessed video tensor of shape (1, C, T, H, W).
        device (torch.device): Device to run the model on.

    Returns:
        int: Predicted class index.
    """
    video_tensor = video_tensor.to(device)
    with torch.no_grad():
        outputs = model(video_tensor)  # Forward pass
        _, predicted_class = torch.max(outputs, 1)  # Get the predicted class index
    return predicted_class.item()


In [18]:
# Define your class label mapping
class_labels = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", 
                "Fighting", "RoadAccidents", "Robbery", "Shooting", 
                "Shoplifting", "Stealing", "Vandalism"]

# Example usage
video_path = "output_folder_video\RoadAccidents133_x264.avi"  # Replace with the path to your video file
sequence_length = 16
video_tensor = preprocess_video(video_path, sequence_length)
predicted_class = predict_video(model, video_tensor, device)

print(f"Predicted Class Index: {predicted_class}")
print(f"Predicted Class Label: {class_labels[predicted_class]}")


Predicted Class Index: 6
Predicted Class Label: RoadAccidents


<>:7: SyntaxWarning: invalid escape sequence '\R'
<>:7: SyntaxWarning: invalid escape sequence '\R'
C:\Users\adars\AppData\Local\Temp\ipykernel_25488\3170231067.py:7: SyntaxWarning: invalid escape sequence '\R'
  video_path = "output_folder_video\RoadAccidents133_x264.avi"  # Replace with the path to your video file


Full Workflow

In [ ]:
# 1. Load the saved model
checkpoint_path = "path_to_saved_checkpoint.pth"
checkpoint = torch.load(checkpoint_path)
num_classes = 12
model = r3d_18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 2. Preprocess the video
video_path = "path_to_video.mp4"  # Path to your input video
sequence_length = 16
video_tensor = preprocess_video(video_path, sequence_length)

# 3. Infer the output
predicted_class = predict_video(model, video_tensor, device)

# 4. Map index to label
class_labels = ["Abuse", "Arson", "Assault", "Burglary", "Explosion", 
                "Fighting", "RoadAccidents", "Robbery", "Shooting", 
                "Shoplifting", "Stealing", "Vandalism"]
print(f"Predicted Class Index: {predicted_class}")
print(f"Predicted Class Label: {class_labels[predicted_class]}")
